# **VASTNet Training**

Training Yolov3-tiny on a custom dataset (ENA24 Wildlife Captures)

The dataset we use is [ENA24 Wildlife Captures](https://lila.science/datasets/ena24detection).

Weights that have been pretrained on ImageNet can be downloaded here. 

```wget https://pjreddie.com/media/files/darknet53.conv.74```

## **What you'll need**
In your Google Drive, set up the following folders: 
- cfg/
  - custom.cfg (specify model config)
  - custom.data (specify file locations)
  - custom.names (list of classes)
  - train.txt (pointers to training images)
  - test.txt (pointers to testing images)
  - custom.json (annotations) 
- dataset/
  - Each image should have an associated .jpg file and an associated .txt file in COCO annotation format. 
  The text file should be in the form 



## **Notes**
Make sure to periodically save/download your weights so that no data is lost in the case that notebook execution is interrupted. 

To continuously refresh, right click "Inspect" and in the Console, paste the following snippet: 

```
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
```


## 1. Setting up Darknet

Note that we are cloning the darknet from [AlexeyAB/darknet](https://github.com/AlexeyAB/darknet.git) which is a fork from official [pjreddie/darknet](https://github.com/pjreddie/darknet) repository. It contains the code for multiple YoloV3 and YoloV4 modifications and is actively maintained.

In [1]:
# Clone Darknet repo 
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15376, done.
remote: Total 15376 (delta 0), reused 0 (delta 0), pack-reused 15376
Receiving objects: 100% (15376/15376), 13.98 MiB | 12.95 MiB/s, done.
Resolving deltas: 100% (10341/10341), done.


In [2]:
# Build Darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (itera

## 2. Downloading and preprocessing dataset

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
  # Copy cfg, images/labels from Google Drive, and create backup directory for saving weights 
  !cp -rf '/content/drive/My Drive/VASTNet/small_cfg' '/content/small_cfg'           # Cfg files
  !cp -rf '/content/drive/My Drive/VASTNet/small_dataset' '/content/small_dataset'   # Images and labels
  !cp -rf '/content/drive/My Drive/VASTNet/small_best.weights' '/content/small_best.weights'   # Images and labels
  !cp -rf '/content/drive/My Drive/VASTNet/small_dataset' '/content/small_dataset'   # Images and labels
!mv '/content/small_dataset' '/content/darknet'
  # Pretrained weights 
  # !cp -rf '/content/drive/My Drive/VASTNet/darknet53.conv.74' '/content/darknet53.conv.74'
  # Make backup directory
  # !mkdir '/content/small_backup'

### 3.1 Downloading the weights

Download weights for YoloV3-tiny and for YoloV4-tiny (Skip if using custom weights)

In [ ]:
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
# !wget https://github.com/GotG/yolotinyv3_medmask_demo/raw/master/yolov3-tiny.conv.15

## 4. Training

We set up all the things that we need for training.

As we are training with `-map` flag, we will get the best weights saved in directory with suffix *best*. Those will be the weights that give the highest mAP on the validation set, in order to prevent overfitting.

![overfitting-map](https://camo.githubusercontent.com/d7f220e15ca2f3a801b998cd0c48d88f5029ccf0d9479a43c48a25f87bd5f1e6/68747470733a2f2f6873746f2e6f72672f66696c65732f3564632f3761652f3766612f35646337616537666164396434653365623361343834633538626663316666352e706e67)

In [5]:
%cd /content/darknet/

/content/darknet


Begin training. 

In [ ]:
!./darknet detector train /content/small_cfg/small.data \
                          /content/small_cfg/small.cfg \
                          /content/small_backup/small_final.weights \
                          -dont_show -ext_output -map -clear

Streaming output truncated to the last 5000 lines.
 total_bbox = 53631, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.762330), count: 7, class_loss = 0.094437, iou_loss = 0.182497, total_loss = 0.276934 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.817670), count: 1, class_loss = 0.014767, iou_loss = 0.050193, total_loss = 0.064961 
 total_bbox = 53639, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.733593), count: 5, class_loss = 0.138139, iou_loss = 0.229488, total_loss = 0.367627 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.686555), count: 3, class_loss = 0.094729, iou_loss = 0.237742, total_loss = 0.332471 
 total_bbox = 53647, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.842873), count: 7, class_loss = 0.279270, iou

## 6. Testing and evaluation

We can now use the best weights to evaluate the model on the test data set.

In [ ]:
weights_best = "/content/small_backup/small_best.weights"

### 6.1 Testing on a random image

First, we will show how the model work on a randomly selected image from our data. We test the model and visualize the image.

In [ ]:
!./darknet detector test /content/small_cfg/small.data  /content/small_cfg/small.cfg  $weights_best /content/small_dataset/4860.jpg -ext_output


 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
Couldn't open file: /content/obj.data


The prediction was saved to *predictions.jpg*.

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

img = cv2.imread("predictions.jpg")
cv2_imshow(img)

AttributeError: ignored

### 6.2 Evaluation on a test data set

As Yolo does not come with a default code for evaluation on custom data sets, we have to use the second `obj_test.data` file that we made, where we set `valid` to `test.txt`. We can use `map` with our config file and best weights to run the evaluation on the test set.

In [ ]:
!./darknet detector map /content/small_cfg/small.data /content/small_cfg/small.cfg $weights_best -points 0

In the last few lines you can see the Precision, Recall, and F1 score, as well as TP, FP, FN, and average IoU.

## 7. Saving model weights

Finally, we will download the weights.

In [ ]:
from google.colab import files
files.download(weights_best)